# 6.888 Final Project - Winograd Transforms

### Madeleine Waller & Diana Wofk

In this project, we use as our baseline this course's lab 4 WS 2D Architecture.  We build on this architecture in the following steps: <br />
- Include the baseline architecture with added energy computations for performance comparison.
- Redesign the PE array to perform element wise multiplications of the transformed weights/ifmaps (rather than convolution). Implement off chip pre/post processing of ifmaps, weights, and ofmaps.
- Move the post processing (reverse transform of ofmaps) onto the chip via a specialized ALU array, similar to the PE array.
- Move the pre processing on chip (in addition to the post processing), using a similar specialized ALU array to transform both the weights and ifmaps.

In [1]:
from nnsim.simulator import run_tb

%load_ext autoreload
%autoreload 2

## Baseline WS 2D Architecture

In [2]:
from ws_2d.tb import WSArchTB

ws_tb = WSArchTB()
run_tb(ws_tb, verbose=False, dump_stats=True)

ifmap glb depth: 16
psum glb depth: 32
weight glb depth: 0
ifmap:  [[[  3 -10  15  11]
  [ -6  -3   3  -2]
  [-20  10  -2 -11]
  [ -5  23  10   0]]

 [[  7   0   8  -9]
  [  0   1  10 -16]
  [ -6 -20  -6  -7]
  [ -6  13 -16   9]]

 [[  1  -4   1   5]
  [ -1  -1   8  -8]
  [  6  -5  -3  -3]
  [ -5   1   9   3]]

 [[-16 -11   0   0]
  [  6  13  -2  24]
  [  3  15   0 -19]
  [  0   2  -2  11]]]
weights:  [[[[  5   3   8   2   4  -8   4   3]
   [  3   2 -20  13   0 -13  12  -2]
   [  7  -8  -8  -6  23  -2   3   0]
   [  0  10   2   1  -3  -8  18 -12]]

  [[  0  -1  19   5  -1   8  -7   1]
   [  1  -6  -3 -16   2  -8  12  17]
   [-29 -10 -10  -1  -7   1   0  -5]
   [  9   1  -4  -8   0   8 -16  -6]]

  [[-18   0   5 -25   3  17   5  -4]
   [  4  -1   0   8  -4  -7  -1   5]
   [-21   4  -2   2   1   9  15   6]
   [  0 -17 -16  -1   2 -24   4   3]]]


 [[[ -1  -1  16  -3  -1  -2  -4  10]
   [ 28  -3   0  -8  -8  10   3   1]
   [  4  15  15 -11  16 -10   0   8]
   [-15   2  -2 -18   9  14 -17 

## Winograd Transform - Pre/Post Processing Off Chip

In [3]:
from ws_2d_winograd_off.tb import WSArchTB

ws_tb = WSArchTB()
run_tb(ws_tb, verbose=False, dump_stats=True)

ifmap glb depth: 64
weight glb depth: 0
ifmaps:  [[[ 17  -5  -4  -6]
  [ -8 -12   1  -6]
  [-13   1 -14  -9]
  [  4 -12  18  -5]]

 [[  4  -4   0  10]
  [-21  -6  11  11]
  [  0  -7   1 -15]
  [-14 -11  10 -15]]

 [[  4   0  -6  -1]
  [ -4  11   4   5]
  [  0  18   7   0]
  [  4  11 -11  -4]]

 [[ 12  12 -10  -5]
  [ 11   2  15  -6]
  [-15 -11  -2   5]
  [  1 -11   6   1]]]
weights:  [[[[-11  14 -15 -13   0  -8  13  -8]
   [-12 -10   0   3   5   0  11   5]
   [ 12   2  -2 -12  -6  -7  -3  11]
   [  9 -20   0  -4  -9   9  -1  -5]]

  [[ 16   5   2   3 -13 -12   9  -5]
   [  0 -11   3   2  10  -1   6   8]
   [-15  -5 -10   2   3   5  -3 -11]
   [-12  -1  11 -20 -15  -4   4  11]]

  [[  6  -8   3   9   5   5  -6   6]
   [-18  -2  19  -1  25   9  -9  10]
   [-10 -11  -3   5   4  20  11  16]
   [ -2   1 -19  11  -4  15   5  -5]]]


 [[[ 13  -4   5  18   0   5  -7   6]
   [ -5  11  19 -22 -12  -9   3  -3]
   [  5  -6   8  13  -2  -3   0   5]
   [  7  -7  -7  -3  16   3   4  -9]]

  [[  9  -6

## Winograd Transform - Move Post Processing On Chip

In [4]:
from ws_2d_winograd_post_on.tb import WSArchTB

ws_tb = WSArchTB()
run_tb(ws_tb, verbose=False, dump_stats=True)

ifmap glb depth: 64
weight glb depth: 0
ifmaps:  [[[  3   4  -3  -6]
  [ -8 -19   2 -13]
  [  3 -10  -1   5]
  [ -8  18   3 -10]]

 [[  5   4 -15  -8]
  [  0  -3  24  -5]
  [-19   7   3   5]
  [ -6  -3   0   1]]

 [[ 13  -5 -13  -2]
  [ 10  -2 -17   3]
  [ -6  -3  -5  -5]
  [-15  -8  -8   2]]

 [[ 20 -17   0  -6]
  [ -8   2   4  -8]
  [ -8  15  12  -9]
  [  3  15   7  16]]]
weights:  [[[[ -9   2  10  13  11   0  10  -2]
   [-13 -11  -5   3  -5  -7   0   0]
   [  3  14  -1 -13   0   9  15  -7]
   [ -6  10  -3   7 -11 -10   2  -9]]

  [[ 10   0  -1  -9  -3   8  -1  -3]
   [ -2   9 -14   7 -17  -2   3   5]
   [ -6  11   4  14   0   4  15   3]
   [ -2   7   5   0  -7  -3 -16 -15]]

  [[  7 -25   2   0  14  -2   0 -14]
   [  0 -24  -2  -1   2   5 -10  -2]
   [ -5  -1  -4  16  12  -4  -7  -5]
   [-12   4  -4  -6   2 -11   2 -21]]]


 [[[ -1  14  21  -3   6  11   5 -13]
   [  8   0 -26   5  14  24   4   0]
   [ -3  12  -3   1  -1   1   7   0]
   [  0  -5 -11   5   0 -11  -9  -7]]

  [[ -6   2

## Winograd Transform - Pre & Post Processing On Chip

In [5]:
from ws_2d_winograd_on_chip.tb import WSArchTB

ws_tb = WSArchTB()
run_tb(ws_tb, verbose=False, dump_stats=True)

ifmap glb depth: 64
weight glb depth: 0
ifmaps:  [[[-17  -1 -10   7]
  [ -6  -6  -9  16]
  [  5  16   7  12]
  [  2 -13   1   7]]

 [[  2  32  -3   0]
  [  4 -11   3   2]
  [  4  -7  22 -15]
  [ -5  -8 -14   6]]

 [[-12  -7   5   7]
  [  7  18  -2 -10]
  [ -9   5  -3  21]
  [-15   7   3   2]]

 [[ -8 -22 -17   4]
  [  7  24  -7  -2]
  [  1  -5   5  -1]
  [ 13   6 -14   0]]]
weights:  [[[[  1  -3   1  -7  -5  -7  -6  10]
   [ -7  -5   2  -3  -2 -10  -8  -5]
   [-17 -16  13  -1  12   3  -2  -5]
   [ 20  -5   9   1  -8   2  -9  -2]]

  [[  3  -3  14   0   9 -15 -18 -10]
   [-13  -6 -13  -3   1 -12   5   4]
   [  5   4   4 -10 -15  -9  -6 -10]
   [ -9  -7  13  13  11  14  -1  -7]]

  [[ 12   2  -1  20  -7  -3 -10  -1]
   [ -2   8  -7   0  -4  13  10   4]
   [ -2  -9  17  14   0  12   0  -7]
   [  2 -17   6  -3 -12   1   7   0]]]


 [[[  6   5 -15   9   4   8   3  -3]
   [ -7  -7  -7   3   4  -2  -4  -4]
   [ -2   3  -2   8  -4   2   0  13]
   [  0   0  -9   0  -5  13 -11  -8]]

  [[ 15  -1